In [37]:
import pandas_datareader.data as web
import pandas as pd
from pathlib import Path

In [53]:
financial_markets = ['^RUT', '^VIX']
commodity_market_futures = ['GC=F', 'CL=F']
economic_env = ['^TNX', '^FVX']
start_date= '2016-01-01'
end_date= '2022-06-15'

In [54]:
def data_maker(tickers, start_date, end_date):
    df = pd.DataFrame({})
    for ticker in tickers:
        data = web.DataReader(ticker, 'yahoo', start=start_date, end=end_date)
        data['ticker'] = ticker
        data = data[['ticker', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close']]
        df = pd.concat([df, data])
    return df

In [55]:
fin_mkts_df = data_maker(financial_markets, start_date=start_date, end_date=end_date)
com_mkts_df = data_maker(commodity_market_futures, start_date=start_date, end_date=end_date)
econ_env_df = data_maker(economic_env, start_date=start_date, end_date=end_date)

In [58]:
fin_mkts_df['daily_change'] = fin_mkts_df.Close.pct_change()
com_mkts_df['daily_change'] = com_mkts_df.Close.pct_change()
econ_env_df['daily_change'] = econ_env_df.Close.pct_change()